In [138]:
import pandas as pd
import numpy as np

from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

# display for this notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [118]:
import os
os.getcwd()

'/Users/Bin/repos/competitions/O2O-Coupon-Usage-Forecast'

In [119]:
# import data
dfoff = pd.read_csv('../../ml-datasets/O2O-Coupon-Usage-Forecast/ccf_offline_stage1_train.csv')
dfon = pd.read_csv('../../ml-datasets/O2O-Coupon-Usage-Forecast/ccf_online_stage1_train.csv')
dftest = pd.read_csv('../../ml-datasets/O2O-Coupon-Usage-Forecast/ccf_offline_stage1_test_revised.csv')

In [120]:
dfoff.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date
count,1.754884e+06,1.754884e+06,1.053282e+06,1.648881e+06,1.053282e+06,7.769840e+05
mean,3.689255e+06,4.038808e+03,6.815398e+03,2.361636e+00,2.016032e+07,2.016040e+07
std,2.123428e+06,2.435963e+03,4.174276e+03,3.483974e+00,1.771416e+02,1.621500e+02
min,4.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.016010e+07,2.016010e+07
25%,1.845052e+06,1.983000e+03,2.840000e+03,0.000000e+00,2.016013e+07,2.016031e+07
50%,3.694446e+06,3.532000e+03,7.430000e+03,0.000000e+00,2.016032e+07,2.016042e+07
75%,5.528759e+06,6.329000e+03,1.032300e+04,3.000000e+00,2.016051e+07,2.016052e+07
max,7.361032e+06,8.856000e+03,1.404500e+04,1.000000e+01,2.016062e+07,2.016063e+07


In [121]:
dfoff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
User_id          int64
Merchant_id      int64
Coupon_id        float64
Discount_rate    object
Distance         float64
Date_received    float64
Date             float64
dtypes: float64(4), int64(2), object(1)
memory usage: 93.7+ MB


In [122]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [123]:
dfoff.sample(10)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
1234663,1795047,6434,10050.0,50:5,6.0,20160125.0,NaN
970904,4177411,7113,NaN,NaN,0.0,NaN,20160303.0
818473,1280626,2934,NaN,NaN,0.0,NaN,20160413.0
1708784,1773962,8370,NaN,NaN,0.0,NaN,20160427.0
1618706,3002069,1469,2902.0,0.95,0.0,20160517.0,NaN
765352,3894924,1379,NaN,NaN,3.0,NaN,20160426.0
363843,7032674,351,NaN,NaN,0.0,NaN,20160625.0
1092630,6455571,1427,NaN,NaN,0.0,NaN,20160530.0
614413,729061,1469,NaN,NaN,10.0,NaN,20160205.0
1248902,3013606,129,NaN,NaN,0.0,NaN,20160220.0


In [124]:
dfoff.isna().sum()

User_id               0
Merchant_id           0
Coupon_id        701602
Discount_rate    701602
Distance         106003
Date_received    701602
Date             977900
dtype: int64

In [125]:
dfoff.isnull().sum()

User_id               0
Merchant_id           0
Coupon_id        701602
Discount_rate    701602
Distance         106003
Date_received    701602
Date             977900
dtype: int64

In [126]:
print('有优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'].notna()) & (dfoff['Date'].notna())].shape[0])
print('有优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'].notna()) & (dfoff['Date'].isna())].shape[0])
print('无优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'].isna()) & (dfoff['Date'].notna())].shape[0])
print('无优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'].isna()) & (dfoff['Date'].isna())].shape[0])

有优惠卷，购买商品：75382
有优惠卷，未购商品：977900
无优惠卷，购买商品：701602
无优惠卷，未购商品：0


## Feature Extract

### 1. 打折率（Discount_rate）

In [127]:
print(dfoff['Discount_rate'].unique())

[nan '150:20' '20:1' '200:20' '30:5' '50:10' '10:5' '100:10' '200:30'
 '20:5' '30:10' '50:5' '150:10' '100:30' '200:50' '100:50' '300:30'
 '50:20' '0.9' '10:1' '30:1' '0.95' '100:5' '5:1' '100:20' '0.8' '50:1'
 '200:10' '300:20' '100:1' '150:30' '300:50' '20:10' '0.85' '0.6' '150:50'
 '0.75' '0.5' '200:5' '0.7' '30:20' '300:10' '0.2' '50:30' '200:100'
 '150:5']


根据打印的结果来看，打折率分为 3 种情况：
* ‘null’ 表示没有打折
* [0,1] 表示折扣率
* x:y 表示满 x 减 y

那我们的处理方式可以构建 4 个函数，分别提取 4 种特征，分别是：

* 打折类型：getDiscountType()
* 折扣率：convertRate()
* 满多少：getDiscountMan()
* 减多少：getDiscountJian()

In [128]:
def getDiscountType(row):
    if row is np.nan:
        return np.nan
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row is np.nan:
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return (float(rows[0])-float(rows[1])) / float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if row is np.nan:
        return 0
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if row is np.nan:
        return 0
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
   # convert discount_rate
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)

    print(df['discount_rate'].unique())

    return df

In [129]:
dfoff = processData(dfoff)
dftest = processData(dftest)

[1.         0.86666667 0.95       0.9        0.83333333 0.8
 0.5        0.85       0.75       0.66666667 0.93333333 0.7
 0.6        0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]
[0.83333333 0.9        0.96666667 0.8        0.95       0.75
 0.98       0.5        0.86666667 0.6        0.66666667 0.7
 0.85       0.33333333 0.94       0.93333333 0.975      0.99      ]


In [130]:
dfoff.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian
count,1.754884e+06,1.754884e+06,1.053282e+06,1.648881e+06,1.053282e+06,7.769840e+05,1.053282e+06,1.754884e+06,1.754884e+06,1.754884e+06
mean,3.689255e+06,4.038808e+03,6.815398e+03,2.361636e+00,2.016032e+07,2.016040e+07,9.684111e-01,9.065689e-01,4.707850e+01,6.226428e+00
std,2.123428e+06,2.435963e+03,4.174276e+03,3.483974e+00,1.771416e+02,1.621500e+02,1.749030e-01,1.035167e-01,6.801246e+01,8.730104e+00
min,4.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.016010e+07,2.016010e+07,0.000000e+00,2.000000e-01,0.000000e+00,0.000000e+00
25%,1.845052e+06,1.983000e+03,2.840000e+03,0.000000e+00,2.016013e+07,2.016031e+07,1.000000e+00,8.333333e-01,0.000000e+00,0.000000e+00
50%,3.694446e+06,3.532000e+03,7.430000e+03,0.000000e+00,2.016032e+07,2.016042e+07,1.000000e+00,9.000000e-01,2.000000e+01,5.000000e+00
75%,5.528759e+06,6.329000e+03,1.032300e+04,3.000000e+00,2.016051e+07,2.016052e+07,1.000000e+00,1.000000e+00,1.000000e+02,1.000000e+01
max,7.361032e+06,8.856000e+03,1.404500e+04,1.000000e+01,2.016062e+07,2.016063e+07,1.000000e+00,1.000000e+00,3.000000e+02,1.000000e+02


### 2. 距离（Distance）
距离字段表示用户与商店的地理距离，显然，距离的远近也会影响到优惠券的使用与否。那么，我们就可以把距离也作为一个特征。首先看一下距离有哪些特征值：

In [131]:
print(dfoff['Distance'].unique())

[ 0.  1. nan  2. 10.  4.  7.  9.  3.  5.  6.  8.]


In [132]:
# convert distance
dfoff['distance'] = dfoff['Distance'].fillna(-1).astype(int)
print(dfoff['distance'].unique())
dftest['distance'] = dftest['Distance'].fillna(-1).astype(int)
print(dftest['distance'].unique())

[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


In [133]:
dfoff.describe()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian,distance
count,1.754884e+06,1.754884e+06,1.053282e+06,1.648881e+06,1.053282e+06,7.769840e+05,1.053282e+06,1.754884e+06,1.754884e+06,1.754884e+06,1.754884e+06
mean,3.689255e+06,4.038808e+03,6.815398e+03,2.361636e+00,2.016032e+07,2.016040e+07,9.684111e-01,9.065689e-01,4.707850e+01,6.226428e+00,2.158577e+00
std,2.123428e+06,2.435963e+03,4.174276e+03,3.483974e+00,1.771416e+02,1.621500e+02,1.749030e-01,1.035167e-01,6.801246e+01,8.730104e+00,3.470772e+00
min,4.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.016010e+07,2.016010e+07,0.000000e+00,2.000000e-01,0.000000e+00,0.000000e+00,-1.000000e+00
25%,1.845052e+06,1.983000e+03,2.840000e+03,0.000000e+00,2.016013e+07,2.016031e+07,1.000000e+00,8.333333e-01,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.694446e+06,3.532000e+03,7.430000e+03,0.000000e+00,2.016032e+07,2.016042e+07,1.000000e+00,9.000000e-01,2.000000e+01,5.000000e+00,0.000000e+00
75%,5.528759e+06,6.329000e+03,1.032300e+04,3.000000e+00,2.016051e+07,2.016052e+07,1.000000e+00,1.000000e+00,1.000000e+02,1.000000e+01,3.000000e+00
max,7.361032e+06,8.856000e+03,1.404500e+04,1.000000e+01,2.016062e+07,2.016063e+07,1.000000e+00,1.000000e+00,3.000000e+02,1.000000e+02,1.000000e+01


In [134]:
dfoff['Date'].isna().sum()

977900

### 3. 领劵日期（Date_received）
是还有一点很重要的是领券日期，因为一般而言，周末领取优惠券去消费的可能性更大一些。因此，我们可以构建关于领券日期的一些特征：

* weekday : {null, 1, 2, 3, 4, 5, 6, 7}
* weekday_type : {1, 0}（周六和周日为1，其他为0）
* Weekday_1 : {1, 0, 0, 0, 0, 0, 0}
* Weekday_2 : {0, 1, 0, 0, 0, 0, 0}
* Weekday_3 : {0, 0, 1, 0, 0, 0, 0}
* Weekday_4 : {0, 0, 0, 1, 0, 0, 0}
* Weekday_5 : {0, 0, 0, 0, 1, 0, 0}
* Weekday_6 : {0, 0, 0, 0, 0, 1, 0}
* Weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [135]:
def getWeekday(row):
    if row == 'nan':
        return np.nan
    elif row is np.nan:
        return np.nan
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_tpye'] = dfoff['weekday'].apply(lambda x: 1 if x in [6, 7] else 0)
dftest['weekday_tpye'] = dftest['weekday'].apply(lambda x: 1 if x in [6, 7] else 0)

# change weekday to one-hot encoding
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]

tmpdf = pd.get_dummies(dfoff['weekday'])
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'])
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

### 标注标签 Label

有了特征之后，我们还需要对训练样本进行 label 标注，即确定哪些是正样本（y = 1），哪些是负样本（y = 0）。我们要预测的是用户在领取优惠券之后 15 之内的消费情况。所以，总共有三种情况：

1.Date_received == ‘null’：

表示没有领到优惠券，无需考虑，y = -1

2.(Date_received != ‘null’) & (Date != ‘null’) & (Date – Date_received <= 15)：

表示领取优惠券且在15天内使用，即正样本，y = 1

3.(Date_received != ‘null’) & ((Date == ‘null’) | (Date – Date_received > 15))：

表示领取优惠券未在在15天内使用，即负样本，y = 0

好了，知道规则之后，我们就可以定义标签备注函数了。

In [136]:
def label(row):
    if row['Date_received'] == 'nan':
        return -1
    if row['Date'] != 'nan':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
dfoff['label'] = dfoff.apply(label, axis=1)

KeyboardInterrupt: 

In [ ]:
print(dfoff['label'].value_counts())

很清晰地，正样本共有 64395 例，负样本共有 988887 例。显然，正负样本数量差别很大。这也是为什么会使用 AUC 作为模型性能评估标准的原因。

## 建立模型
接下来就是最主要的建立机器学习模型了。首先确定的是我们选择的特征是上面提取的 14 个特征，为了验证模型的性能，需要划分验证集进行模型验证，划分方式是按照领券日期，即训练集：20160101-20160515，验证集：20160516-20160615。我们采用的模型是简单的 SGDClassifier。

### 1.划分训练集和验证集

In [ ]:
# data split
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < '20160516')].copy()
valid = df[(df['Date_received'] >= '20160516') & (df['Date_received'] <= '20160615')].copy()
print('Train Set: \n', train['label'].value_counts())
print('Valid Set: \n', valid['label'].value_counts())

In [ ]:
# feature
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','distance', 'weekday', 'weekday_type'] + weekdaycols
print('共有特征：',len(original_feature),'个')
print(original_feature)

### 2. 构建模型

In [ ]:
def check_model(data, predictors):
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.

    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=3, shuffle=True)

    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    grid_search = grid_search.fit(data[predictors], data['label'])

    return grid_search

模型采用的是 SGDClassifier，使用了 Python 中的 Pipeline 管道机制，可以使参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。交叉验证采用 StratifiedKFold，其用法类似 Kfold，但是 StratifiedKFold 是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。

### 3.训练

接下来就可以使用该模型对训练集进行训练了，整个训练过程大概 1-2 分钟的时间。

In [ ]:
predictors = original_feature
model = check_model(train, predictors)

### 4.验证

然后对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [137]:
# valid predict
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]
valid1.head(5)

NameError: name 'model' is not defined

注意这里得到的结果 pred_prob 是概率值（预测样本属于正类的概率）。

最后，就可以对验证集计算 AUC。直接调用 sklearn 库自带的计算 AUC 函数即可。

In [139]:
# avgAUC calculation
vg = valid1.groupby(['Coupon_id'])
aucs = []
for i in vg:
   tmpdf = i[1] 
   if len(tmpdf['label'].unique()) != 2:
       continue
   fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
   aucs.append(auc(fpr, tpr))
print(np.average(aucs))

NameError: name 'valid1' is not defined

最终得到的 AUC 就等于 0.53。

## 测试
训练完模型之后，就是使用训练好的模型对测试集进行测试了。并且将测试得到的结果（概率值）按照规定的格式保存成一个 .csv 文件。

In [140]:
# test prediction for submission
y_test_pred = model.predict_proba(dftest[predictors])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
dftest1['Probability'] = y_test_pred[:,1]
dftest1.to_csv('submit.csv', index=False, header=False)
dftest1.head(5)

NameError: name 'model' is not defined